# ETL

# Libraries

In [171]:
import os
import ast

import numpy as np
import pandas as pd

# Configurations

In [172]:
pd.set_option('display.max_columns', None)

pd.set_option('display.max_colwidth', None)

pd.set_option('display.width', 1000)

# Extract

Los dos datasets se descargan de esta dirección:

https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5

Los datasets deben estar en una carpeta llamada `raw` ubicada en la raiz del proyecto.

In [173]:
ruta_raw = os.path.abspath('../raw/')

## `movies`

In [174]:
movies = pd.read_csv(
    os.path.join(
        ruta_raw, 'movies_dataset.csv'))

C:\Users\franc\AppData\Local\Temp\ipykernel_20768\2227159362.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(


In [175]:
movies.head().T

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",NaN,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]","[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.","A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.","Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.","Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."


## `credits`

In [176]:
credits = pd.read_csv(
    os.path.join(
        ruta_raw, 'credits.csv'))

In [177]:
credits.head()

cast  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYG

# Transform

Transformaciones a realizar:

- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- Los valores nulos del campo release date deben eliminarse.

- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

## `movies`

In [178]:
movies.head(1).T

,0
adult,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
budget,30000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
homepage,http://toystory.disney.com/toy-story
id,862
imdb_id,tt0114709
original_language,en
original_title,Toy Story
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."


### Unnesting

#### belongs_to_collection

In [179]:
belongs_to_collection = movies[['id', 'belongs_to_collection']].copy()

movies = movies.drop(columns=['belongs_to_collection'])

In [180]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     45466 non-null  object
 1   belongs_to_collection  4494 non-null   object
dtypes: object(2)
memory usage: 710.5+ KB


In [181]:
belongs_to_collection.head(5)

,id,belongs_to_collection
0,862,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
1,8844,NaN
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"
3,31357,NaN
4,11862,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"


In [182]:
belongs_to_collection.rename(
    columns={
        'id': 'movie_id',
        'belongs_to_collection': 'collection_name'
    },
    inplace=True)

In [183]:
belongs_to_collection.head(5)

,movie_id,collection_name
0,862,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
1,8844,NaN
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"
3,31357,NaN
4,11862,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"


In [184]:
belongs_to_collection['collection_name'] = belongs_to_collection['collection_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

new_columns = pd.json_normalize(belongs_to_collection['collection_name'])

belongs_to_collection = belongs_to_collection.join(new_columns)

belongs_to_collection.drop(columns=['collection_name'], inplace=True)

belongs_to_collection.rename(
    columns={
        'id': 'collection_id',
        'name': 'collection_name'
    },
    inplace=True)

In [185]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         45466 non-null  object 
 1   collection_id    4491 non-null   float64
 2   collection_name  4491 non-null   object 
 3   poster_path      3948 non-null   object 
 4   backdrop_path    3263 non-null   object 
dtypes: float64(1), object(4)
memory usage: 1.7+ MB


In [186]:
belongs_to_collection.head()

,movie_id,collection_id,collection_name,poster_path,backdrop_path
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,8844,NaN,NaN,NaN,NaN
2,15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,31357,NaN,NaN,NaN,NaN
4,11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [187]:
belongs_to_collection['collection_id'].value_counts()

collection_id
415931.0    29
421566.0    27
645.0       26
96887.0     26
37261.0     25
34055.0     22
413661.0    21
374509.0    16
148324.0    15
38451.0     15
425164.0    15
19163.0     14
224026.0    14
479319.0    13
69788.0     13
478628.0    12
397842.0    12
24717.0     12
9735.0      12
41658.0     10
458017.0    10
188138.0    10
107949.0    10
8917.0      10
97919.0     10
25542.0     10
330555.0     9
53043.0      9
94039.0      9
937.0        9
123724.0     9
59235.0      9
115225.0     9
23240.0      9
113589.0     9
418326.0     8
9485.0       8
10.0         8
1241.0       8
65249.0      8
63170.0      8
91361.0      8
101646.0     8
95051.0      8
8581.0       8
161766.0     8
158365.0     8
174218.0     7
269098.0     7
1972.0       7
10441.0      7
17178.0      7
33071.0      7
168880.0     7
227140.0     7
1575.0       7
2704.0       7
89959.0      7
91657.0      7
9338.0       7
374511.0     7
135441.0     7
656.0        7
256377.0     7
196753.0     6
19285.0    

#### genres

In [188]:
genres = movies[['id', 'genres']].copy()

movies = movies.drop(columns=['genres'])

In [189]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45466 non-null  object
 1   genres  45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [190]:
genres.head(5)

,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [191]:
genres.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [192]:
genres.head(5)

,movie_id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [193]:
genres['genres'] = genres['genres'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = genres.explode('genres')

new_columns = pd.json_normalize(exploded['genres'])

genres = exploded.drop(columns=['genres']).reset_index(drop=True).join(new_columns)

In [194]:
genres.head(5)

,movie_id,id,name
0,862,16.0,Animation
1,862,35.0,Comedy
2,862,10751.0,Family
3,8844,12.0,Adventure
4,8844,14.0,Fantasy


In [195]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93548 entries, 0 to 93547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  93548 non-null  object 
 1   id        91106 non-null  float64
 2   name      91106 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [196]:
genres.rename(
    columns={
        'id': 'genre_id',
        'name': 'genre_name'
    },
    inplace=True)

In [197]:
genres.columns

Index(['movie_id', 'genre_id', 'genre_name'], dtype='object')

In [198]:
genres['genre_id'].value_counts()

genre_id
18.0       20265
35.0       13182
53.0        7624
10749.0     6735
28.0        6596
27.0        4673
80.0        4307
99.0        3932
12.0        3496
878.0       3049
10751.0     2770
9648.0      2467
14.0        2313
16.0        1935
10769.0     1622
10402.0     1598
36.0        1398
10752.0     1323
37.0        1042
10770.0      767
11176.0        1
11602.0        1
29812.0        1
2883.0         1
7759.0         1
7760.0         1
7761.0         1
33751.0        1
17161.0        1
18012.0        1
18013.0        1
23822.0        1
Name: count, dtype: int64

#### production_companies

In [199]:
production_companies = movies[['id', 'production_companies']].copy()

movies = movies.drop(columns=['production_companies'])

In [200]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_companies  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [201]:
production_companies.head(5)

,id,production_companies
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"
2,15602,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]"
3,31357,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]"
4,11862,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]"


In [202]:
production_companies.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [203]:
production_companies.columns

Index(['movie_id', 'production_companies'], dtype='object')

In [204]:
production_companies['production_companies'] = production_companies['production_companies'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_companies.explode('production_companies')

new_columns = pd.json_normalize(exploded['production_companies'])

production_companies = exploded.drop(columns=['production_companies']).reset_index(drop=True).join(new_columns)

In [205]:
production_companies.head(5)

,movie_id,name,id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0
2,8844,Teitler Film,2550.0
3,8844,Interscope Communications,10201.0
4,15602,Warner Bros.,6194.0


In [206]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82426 entries, 0 to 82425
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  82426 non-null  object 
 1   name      70545 non-null  object 
 2   id        70545 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.9+ MB


In [207]:
production_companies.rename(
    columns={
        'id': 'company_id', 
        'name': 'company_name'
    },
    inplace=True)

In [208]:
production_companies.columns

Index(['movie_id', 'company_name', 'company_id'], dtype='object')

In [209]:
production_companies['company_name'].value_counts()

company_name
Warner Bros.                                                                                             1250
Metro-Goldwyn-Mayer (MGM)                                                                                1076
Paramount Pictures                                                                                       1003
Twentieth Century Fox Film Corporation                                                                    836
Universal Pictures                                                                                        830
Columbia Pictures Corporation                                                                             448
Canal+                                                                                                    438
Columbia Pictures                                                                                         431
RKO Radio Pictures                                                                                        2

#### production_countries

In [210]:
production_countries = movies[['id', 'production_countries']].copy()

movies = movies.drop(columns=['production_countries'])

In [211]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_countries  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [212]:
production_countries.head(5)

,id,production_countries
0,862,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
2,15602,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
3,31357,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
4,11862,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"


In [213]:
production_countries.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [214]:
production_countries.columns

Index(['movie_id', 'production_countries'], dtype='object')

In [215]:
production_countries['production_countries'] = production_countries['production_countries'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_countries.explode('production_countries')

new_columns = pd.json_normalize(exploded['production_countries'])

production_countries = exploded.drop(columns=['production_countries']).reset_index(drop=True).join(new_columns)

In [216]:
production_countries.head(5)

,movie_id,iso_3166_1,name
0,862,US,United States of America
1,8844,US,United States of America
2,15602,US,United States of America
3,31357,US,United States of America
4,11862,US,United States of America


In [217]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55711 entries, 0 to 55710
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    55711 non-null  object
 1   iso_3166_1  49423 non-null  object
 2   name        49423 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [218]:
production_countries.rename(
    columns={
        'name': 'country_name'
    },
    inplace=True)

In [219]:
production_countries.columns

Index(['movie_id', 'iso_3166_1', 'country_name'], dtype='object')

In [220]:
production_countries['country_name'].value_counts()

country_name
United States of America                21153
United Kingdom                           4094
France                                   3940
Germany                                  2254
Italy                                    2169
Canada                                   1765
Japan                                    1648
Spain                                     964
Russia                                    912
India                                     828
Hong Kong                                 596
Sweden                                    588
Australia                                 570
South Korea                               495
Belgium                                   447
Denmark                                   386
Finland                                   383
Netherlands                               375
China                                     372
Mexico                                    329
Poland                                    319
Brazil               

#### spoken_languages

In [221]:
spoken_languages = movies[['id', 'spoken_languages']].copy()

movies = movies.drop(columns=['spoken_languages'])

In [222]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                45466 non-null  object
 1   spoken_languages  45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [223]:
spoken_languages.head(5)

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]"
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]"


In [224]:
spoken_languages.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [225]:
spoken_languages.columns

Index(['movie_id', 'spoken_languages'], dtype='object')

In [226]:
spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = spoken_languages.explode('spoken_languages')

new_columns = pd.json_normalize(exploded['spoken_languages'])

spoken_languages = exploded.drop(columns=['spoken_languages']).reset_index(drop=True).join(new_columns)

In [227]:
spoken_languages.head(5)

,movie_id,iso_639_1,name
0,862,en,English
1,8844,en,English
2,8844,fr,Français
3,15602,en,English
4,31357,en,English


In [228]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57135 entries, 0 to 57134
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   57135 non-null  object
 1   iso_639_1  53300 non-null  object
 2   name       53300 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [229]:
spoken_languages.rename(
    columns={
        'name': 'language_name'
    },
    inplace=True)

In [230]:
spoken_languages.columns

Index(['movie_id', 'iso_639_1', 'language_name'], dtype='object')

In [231]:
spoken_languages['language_name'].value_counts()

language_name
English             28745
Français             4196
Deutsch              2625
Español              2413
Italiano             2367
日本語                  1758
Pусский              1563
普通话                   790
हिन्दी                707
Português             591
svenska               560
한국어/조선말               542
Polski                524
                      498
广州话 / 廣州話             473
suomi                 371
Magyar                360
العربية               341
Nederlands            340
No Language           319
Dansk                 302
Český                 284
Türkçe                247
עִבְרִית              215
ελληνικά              213
ภาษาไทย               176
Norsk                 171
فارسی                 142
Latin                 140
Română                128
தமிழ்                 111
Srpski                108
తెలుగు                 68
Tiếng Việt             61
Íslenska               60
اردو                   55
Український            53
Hrvatski               5

In [232]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   homepage           7782 non-null   object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   overview           44512 non-null  object 
 8   popularity         45461 non-null  object 
 9   poster_path        45080 non-null  object 
 10  release_date       45379 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

### Fill in the revenue and budget fields with 0

In [233]:
movies['revenue'].value_counts()

revenue
0.000000e+00    38052
1.200000e+07       20
1.000000e+07       19
1.100000e+07       19
2.000000e+06       18
6.000000e+06       17
5.000000e+06       14
5.000000e+05       13
8.000000e+06       13
1.400000e+07       12
1.000000e+00       12
7.000000e+06       11
3.000000e+06       10
2.000000e+07       10
1.000000e+06       10
3.000000e+00        9
4.000000e+06        9
1.500000e+06        9
2.500000e+06        8
1.500000e+07        8
3.000000e+07        8
4.100000e+06        8
1.600000e+07        8
2.500000e+07        8
1.300000e+07        7
1.800000e+07        7
4.300000e+06        7
9.000000e+06        6
1.300000e+06        6
1.000000e+08        6
1.400000e+06        6
1.900000e+07        5
5.900000e+06        5
8.000000e+00        5
2.200000e+07        5
1.500000e+01        5
3.200000e+06        5
2.800000e+07        5
4.500000e+06        5
1.020000e+08        5
1.700000e+07        5
2.500000e+01        5
7.800000e+06        5
3.600000e+07        5
5.000000e+00        5
2.

In [234]:
movies['budget'].value_counts()

budget
0                                   36573
5000000                               286
10000000                              259
20000000                              243
2000000                               242
15000000                              226
3000000                               223
25000000                              206
1000000                               197
30000000                              190
4000000                               181
6000000                               174
12000000                              172
40000000                              167
8000000                               155
500000                                142
35000000                              141
7000000                               137
50000000                              124
1500000                               120
60000000                              110
18000000                              105
3500000                               103
2500000                    

In [235]:
display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(6)

np.int64(0)

In [236]:
movies[['revenue', 'budget']] = movies[['revenue', 'budget']].fillna(0)

display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(0)

np.int64(0)

### Remove null values ​​from the release_date field

In [237]:
movies['release_date'].isna().sum()

np.int64(87)

In [238]:
movies = movies.dropna(subset=['release_date'])

In [239]:
movies['release_date'].isna().sum()

np.int64(0)

### Format YYYY-mm-dd and create the release_year column

In [240]:
movies['release_date'].head(5)

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [241]:
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [242]:
movies['release_date'].isna().sum()

np.int64(3)

In [243]:
movies[movies['release_date'].isna()]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'JP', 'name': 'Japan'}]",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
movies = movies.dropna(subset=['release_date'])

movies['release_date'].isna().sum()

np.int64(0)

In [245]:
movies['release_year'] = pd.to_datetime(movies['release_date']).dt.year

movies['release_year'].head(5)

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: int32

### Create return column

In [246]:
movies['revenue'] = pd.to_numeric(movies['revenue']).astype(int)
movies['budget'] = pd.to_numeric(movies['budget']).astype(int)

movies['return'] = np.where((movies['revenue'] == 0) | (movies['budget'] == 0), 0, movies['revenue'] // movies['budget']).astype(int)

display(movies[['revenue', 'budget', 'return']].head(5))
print()
display(movies['return'].value_counts())

,revenue,budget,return
0,373554033,30000000,12
1,262797249,65000000,4
2,0,0,0
3,81452156,16000000,5
4,76578911,0,0


return
0           41600
1            1021
2             810
3             497
4             341
5             210
6             159
7             109
8              97
9              78
10             52
11             45
12             27
14             26
16             23
13             23
17             16
15             15
22             11
19             11
23             10
25             10
21              9
20              8
18              8
29              6
26              6
30              6
35              5
27              5
24              5
39              5
32              5
34              4
101             4
62              4
36              4
41              3
100             3
55              3
28              3
38              3
94              2
53              2
73              2
67              2
40              2
33              2
52              2
89              2
49              2
59              2
84              2
51              2
60              2
108

### Delete unnecessary columns

In [247]:
movies.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'release_year', 'return'], dtype='object')

In [248]:
movies = movies.drop(columns=["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

In [249]:
movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'release_year', 'return'], dtype='object')

## `credits`

In [251]:
credits.head(1).T

0
cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

#### Unnesting